In [2]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
import pandas as pd
import numpy as np
from tqdm import tqdm

from DeepPurpose import utils # , dataset, CompoundPred
from DeepPurpose import DTI as models
from DeepPurpose.utils import *
from DeepPurpose.dataset import *
from rdkit import Chem
from rdkit import DataStructs
#from rdkit.ML.Cluster import Butina
#from rdkit.Chem import Draw
#from rdkit.Chem import rdFingerprintGenerator
#from rdkit.Chem.Draw import SimilarityMaps
import warnings
import numpy as np
import random 
import pandas as pd 
warnings.filterwarnings("ignore")

sdf_file = "all_ligands.sdf"
supplier = Chem.SDMolSupplier(sdf_file, sanitize=False)
# morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=2048)

# Iterate through the molecules and extract SMILES
ligand_id_to_smile_dict = dict()
for mol in supplier:
    if mol is not None:  # Check if molecule is valid
        try:
            # Attempt to sanitize the molecule
            Chem.SanitizeMol(mol)
            # Generate the fingerprint
            smiles = Chem.MolToSmiles(mol)
            id = int(mol.GetProp('GtoPdb_ID'))
            ligand_id_to_smile_dict[id] = smiles
        except Exception as e:
            print(f"Error processing molecule: {e}")

[01:51:27] ERROR: Counts line too short: '' on line4
[01:51:27] ERROR: moving to the beginning of the next molecule
[01:51:28] Explicit valence for atom # 1 N, 4, is greater than permitted


Error processing molecule: Explicit valence for atom # 1 N, 4, is greater than permitted


[01:51:28] Explicit valence for atom # 8 C, 5, is greater than permitted


Error processing molecule: Explicit valence for atom # 8 C, 5, is greater than permitted


[01:51:30] Explicit valence for atom # 0 N, 4, is greater than permitted


Error processing molecule: Explicit valence for atom # 0 N, 4, is greater than permitted


[01:51:30] Explicit valence for atom # 0 O, 3, is greater than permitted


Error processing molecule: Explicit valence for atom # 0 O, 3, is greater than permitted
Error processing molecule: Explicit valence for atom # 15 N, 4, is greater than permitted


[01:51:31] Explicit valence for atom # 15 N, 4, is greater than permitted


In [3]:
dti_df = pd.read_csv('gpcr_interactions.csv', skiprows=1)
uni_to_aac_df = pd.read_csv('GtP_to_AAC_mapping.csv')


dti_df = dti_df[['Target ID', 'Ligand ID', 'Affinity Median']].dropna()
target_ids = dti_df['Target ID'].to_numpy()
ligand_ids = dti_df['Ligand ID'].to_numpy()
ys = dti_df['Affinity Median'].to_numpy()


uni_to_aac_df = uni_to_aac_df[['GtoPdb IUPHAR ID', 'AAC']]
target_id_to_aac_dict = uni_to_aac_df.groupby("GtoPdb IUPHAR ID")["AAC"].first().to_dict()


ligand_smiles_list = []
target_aac_list = []
affinity_list = []

errors = 0
for j in range(len(target_ids)):
    target_id = int(target_ids[j])
    ligand_id = int(ligand_ids[j])
    if ligand_id in ligand_id_to_smile_dict.keys() and \
        target_id in target_id_to_aac_dict.keys():
        ligand_smiles = ligand_id_to_smile_dict[ligand_id]
        target_aac = target_id_to_aac_dict[target_id]
        
        if len(ligand_smiles) > 0 and len(target_aac) > 0:
            ligand_smiles_list.append(ligand_smiles)
            target_aac_list.append(target_aac)
            affinity_list.append(ys[j])
    else:
        errors += 1

In [4]:
def get_unique_inds(lst):
    seen = {}
    indices = []
    
    for i, val in enumerate(lst):
        if val not in seen:
            seen[val] = i
            indices.append(i)

    return sorted(indices)


unique_inds = get_unique_inds(ligand_smiles_list)
print(len(unique_inds))
print(len(ligand_smiles_list))

2923
6335


In [5]:
unique_target_aac_list = [target_aac_list[i] for i in unique_inds]
unique_ligand_smiles_list = [ligand_smiles_list[i] for i in unique_inds]
unique_affinity_list = [affinity_list[i] for i in unique_inds]

In [7]:
drug_encoding = 'CNN'
target_encoding = 'Transformer'

full_data_df = utils.data_process(unique_ligand_smiles_list, unique_target_aac_list, unique_affinity_list, 
                                  drug_encoding, target_encoding, 
                                  split_method='no_split')

num = len(full_data_df)


# isotopes are not distinguished by tanimoto
further_duplicates = []


similar_mat = np.zeros((num, num))
for i in tqdm(range(num)):
    mol_i = Chem.RDKFingerprint(Chem.MolFromSmiles(full_data_df.iloc[i,0]))
    for j in np.arange(i, num):
        mol_j = Chem.RDKFingerprint(Chem.MolFromSmiles(full_data_df.iloc[j,0]))
        similar_mat[i,j] = DataStructs.TanimotoSimilarity(mol_i, mol_j)

        if i != j and similar_mat[i,j] == 1:
            print("--------------------")
            print(f'ERROR for (i,j) = {(i,int(j))}')
            print(full_data_df.iloc[i,0])
            print(full_data_df.iloc[j,0])
            print("--------------------")
            further_duplicates.append(j)

similar_mat = similar_mat + similar_mat.T - np.eye(similar_mat.shape[0])

similar_mat = similar_mat[np.setdiff1d(np.arange(num), further_duplicates)][:,np.setdiff1d(np.arange(num), further_duplicates)]
np.save('similarityMatrix.npy', similar_mat)


Drug Target Interaction Prediction Mode...
in total: 2923 drug-target pairs
encoding drug...
unique drugs: 2923
encoding protein...
unique target sequence: 253
splitting dataset...
do not do train/test split on the data for already splitted data


  5%|▍         | 142/2923 [20:25<4:08:55,  5.37s/it]

--------------------
ERROR for (i,j) = (142, 166)
COc1cc(CC(C)N)c(OC)cc1I
COc1cc(CC(C)N)c(OC)cc1[125I]
--------------------


  9%|▉         | 274/2923 [31:41<3:39:56,  4.98s/it]

--------------------
ERROR for (i,j) = (274, 278)
COc1ccc(NS(=O)(=O)c2ccc(I)cc2)cc1N1CCN(C)CC1
COc1ccc(NS(=O)(=O)c2ccc([125I])cc2)cc1N1CCN(C)CC1
--------------------


...

 99%|█████████▉| 2902/2923 [2:24:56<00:02,  8.86it/s]

--------------------
ERROR for (i,j) = (2900, 2901)
CCC(C)C(NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(Cc1c[nH]cn1)NC(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(C)NC(=O)C(CO)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CCCN=C(N)N)NC(=O)C(C)NC(=O)C(CC(C)C)NC(=O)C(CC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(C)NC(=O)C1CCCN1C(=O)C(C)NC(=O)C(CC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)CNC(=O)C1CCCN1C(=O)C(CC(N)=O)NC(=O)C(CC(=O)O)NC(=O)C1CCCN1C(=O)C(CCCCN)NC(=O)C(CO)NC(=O)C1CCCN1)C(=O)NC(CC(N)=O)C(=O)NC(CC(C)C)C(=O)NC(C(=O)NC(C(=O)NC(CCCN=C(N)N)C(=O)NC(CCC(N)=O)C(=O)NC(CCCN=C(N)N)C(=O)NC(Cc1ccc(O)cc1)C(=O)O)C(C)O)C(C)CC
CCC(C)C(NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(Cc1c[nH]cn1)NC(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(C)NC(=O)C(CO)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CCCN=C(N)N)NC(=O)C(C)NC(=O)C(CC(C)C)NC(=O)C(CC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(C)NC(=O)C1CCCN1C(=O)C(C)NC(=O)C(CC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)CNC(=O)C1CCCN1C(=O)C(CC(N)=O)NC(=O)C(CC(=O)O)NC(=O)C1CCCN1C(=O)C(CCCCN)NC(=O)C(N)CO)C(=O)NC(CC(N)=O)C(=O)NC(CC(C)C)C(=O)N

100%|█████████▉| 2915/2923 [2:24:57<00:00, 17.88it/s]

--------------------
ERROR for (i,j) = (2910, 2911)
CCC(C)C(NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(Cc1c[nH]cn1)NC(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(C)NC(=O)C(CO)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(N)Cc1ccc(O)cc1)C(=O)NC(CC(N)=O)C(=O)NC(CC(C)C)C(=O)NC(C(=O)NC(C(=O)NC(CCCN=C(N)N)C(=O)NC(CCC(N)=O)C(=O)NC(CCCN=C(N)N)C(=O)NC(Cc1ccc(O)cc1)C(=O)O)C(C)O)C(C)CC
CCC(C)C(NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(Cc1c[nH]cn1)NC(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(C)NC(=O)C(N)CO)C(=O)NC(CC(N)=O)C(=O)NC(CC(C)C)C(=O)NC(C(=O)NC(C(=O)NC(CCCN=C(N)N)C(=O)NC(CCC(N)=O)C(=O)NC(CCCN=C(N)N)C(=O)NC(Cc1ccc(O)cc1)C(=O)O)C(C)O)C(C)CC
--------------------


100%|██████████| 2923/2923 [2:24:57<00:00,  2.98s/it]


In [9]:

unique_target_aac_list = [unique_target_aac_list[i] for i in np.setdiff1d(np.arange(len(unique_target_aac_list)), further_duplicates)]
unique_ligand_smiles_list = [unique_ligand_smiles_list[i] for i in np.setdiff1d(np.arange(len(unique_ligand_smiles_list)), further_duplicates)]
unique_affinity_list = [unique_affinity_list[i] for i in np.setdiff1d(np.arange(len(unique_affinity_list)), further_duplicates)]



cleaned_df = pd.DataFrame({
    'target': unique_target_aac_list,
    'ligand': unique_ligand_smiles_list,
    'affinity': unique_affinity_list,
})

cleaned_df.to_csv('./cleaned_drug_data.csv')

In [11]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
import pandas as pd
import numpy as np
from tqdm import tqdm

from DeepPurpose import utils # , dataset, CompoundPred
from DeepPurpose import DTI as models
from DeepPurpose.utils import *
from DeepPurpose.dataset import *
from rdkit import Chem
from rdkit import DataStructs
#from rdkit.ML.Cluster import Butina
#from rdkit.Chem import Draw
#from rdkit.Chem import rdFingerprintGenerator
#from rdkit.Chem.Draw import SimilarityMaps
import warnings
import numpy as np
import sys 
import pandas as pd 
warnings.filterwarnings("ignore")


ligand_smiles_list = unique_ligand_smiles_list
target_aac_list = unique_target_aac_list
affinity_list = unique_affinity_list


for seed in tqdm(range(2500)):
    np.random.seed(seed)

    drug_encoding = 'CNN'
    target_encoding = 'Transformer'


    df = pd.read_csv('cleaned_drug_data.csv')

    affinity_list = df['affinity'].tolist()
    ligand_smiles_list = df['ligand'].tolist()
    target_aac_list = df['target'].tolist()

    n_total = len(affinity_list)
    reind = np.random.permutation(n_total)

    X_drugs_train = [ligand_smiles_list[reind[0:int(n_total*0.75+1)][i]] for i in range(int(n_total*0.75+1))]
    X_targets_train = [target_aac_list[reind[0:int(n_total*0.75+1)][i]] for i in range(int(n_total*0.75+1))]
    y_train = [affinity_list[reind[0:int(n_total*0.75+1)][i]] for i in range(int(n_total*0.75+1))]


    X_drugs_other = [ligand_smiles_list[reind[int(1+n_total*0.75):n_total][i]] for i in range(n_total-int(n_total*0.75+1))]
    X_targets_other = [target_aac_list[reind[int(1+n_total*0.75):n_total][i]] for i in range(n_total-int(n_total*0.75+1))]
    y_other = [affinity_list[reind[int(1+n_total*0.75):n_total][i]] for i in range(n_total-int(n_total*0.75+1))]

    ttrain, tval, ttest = utils.data_process(X_drugs_train, X_targets_train, y_train, 
                                    drug_encoding, target_encoding, 
                                    split_method='random', frac=[0.8,0.2,0.],
                                    random_seed = seed)

    ddata, _, __ = utils.data_process(X_drugs_other, X_targets_other, y_other, 
                                    drug_encoding, target_encoding, 
                                        split_method='random', frac=[1., 0., 0.],
                                        random_seed = seed)


    calib_test_perm = np.random.permutation(len(ddata))


    final_perm = np.array(reind[int(1+n_total*0.75):n_total])[calib_test_perm]


    np.save(f'./permutations/perm_j{seed}.npy', final_perm)

  0%|          | 1/2500 [00:00<08:09,  5.11it/s]

Drug Target Interaction Prediction Mode...
in total: 2156 drug-target pairs
encoding drug...
unique drugs: 2156
encoding protein...
unique target sequence: 243
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 718 drug-target pairs
encoding drug...
unique drugs: 718
encoding protein...
unique target sequence: 187
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 2156 drug-target pairs
encoding drug...
unique drugs: 2156


  0%|          | 2/2500 [00:00<07:20,  5.67it/s]

encoding protein...
unique target sequence: 239
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 718 drug-target pairs
encoding drug...
unique drugs: 718
encoding protein...
unique target sequence: 184
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 2156 drug-target pairs
encoding drug...
unique drugs: 2156
encoding protein...
unique target sequence: 244
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 718 drug-target pairs
encoding drug...
unique drugs: 718


  0%|          | 3/2500 [00:00<07:18,  5.70it/s]

...

unique drugs: 2156
encoding protein...
unique target sequence: 243
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 718 drug-target pairs
encoding drug...
unique drugs: 718
encoding protein...
unique target sequence: 188
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 2156 drug-target pairs
encoding drug...
unique drugs: 2156


100%|█████████▉| 2499/2500 [06:50<00:00,  5.76it/s]

encoding protein...
unique target sequence: 242
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 718 drug-target pairs
encoding drug...
unique drugs: 718
encoding protein...
unique target sequence: 186
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 2156 drug-target pairs
encoding drug...
unique drugs: 2156
encoding protein...
unique target sequence: 245


100%|██████████| 2500/2500 [06:50<00:00,  6.09it/s]

splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 718 drug-target pairs
encoding drug...
unique drugs: 718
encoding protein...
unique target sequence: 183
splitting dataset...
Done.


In [5]:
# from DeepPurpose import utils # , dataset, CompoundPred
# from DeepPurpose import DTI as models
# from DeepPurpose.utils import *
# from DeepPurpose.dataset import *
# from rdkit import Chem
# from rdkit import DataStructs
# #from rdkit.ML.Cluster import Butina
# #from rdkit.Chem import Draw
# #from rdkit.Chem import rdFingerprintGenerator
# #from rdkit.Chem.Draw import SimilarityMaps
# import warnings
# import numpy as np
# import random 
# import pandas as pd 
# warnings.filterwarnings("ignore")
# from tqdm import tqdm

# # load data
# # X_drugs, X_targets, y = dataset.load_process_DAVIS(path = './data', binary = False, convert_to_log = True, threshold = 30)
# # drug_encoding, target_encoding = 'Morgan', 'Conjoint_triad' 
# ligand_smiles_list, target_aac_list, affinity_list = load_process_DAVIS('./data/', binary=False, convert_to_log = True, threshold = 30)
# ligand_smiles_list, target_aac_list, affinity_list \
#     = ligand_smiles_list[:6000], target_aac_list[:6000], affinity_list[:6000]


# drug_encoding = 'MPNN'
# target_encoding = 'CNN'

# full_data_df = utils.data_process(ligand_smiles_list, target_aac_list, affinity_list, 
#                                   drug_encoding, target_encoding, 
#                                   split_method='no_split')


# num = len(full_data_df)


# similar_mat = np.zeros((num, num))
# for i in tqdm(range(num)):
#     mol_i = Chem.RDKFingerprint(Chem.MolFromSmiles(full_data_df.iloc[i,0]))
#     for j in np.arange(i, num):
#         mol_j = Chem.RDKFingerprint(Chem.MolFromSmiles(full_data_df.iloc[j,0]))
#         similar_mat[i,j] = DataStructs.TanimotoSimilarity(mol_i, mol_j)

# similar_mat = similar_mat + similar_mat.T - np.eye(similar_mat.shape[0])
# np.save('similarityMatrix.npy', similar_mat)

Beginning Processing...
Beginning to extract zip file...
Default set to logspace (nM -> p) for easier regression
Done!
Drug Target Interaction Prediction Mode...
in total: 6000 drug-target pairs
encoding drug...
unique drugs: 14
encoding protein...
unique target sequence: 379
splitting dataset...
do not do train/test split on the data for already splitted data


  0%|          | 6/6000 [01:46<29:31:33, 17.73s/it]


KeyboardInterrupt: 